# Data import

data를 가져오는 부분

In [1]:
import csv
import numpy as np

with open('train.csv','r') as dest_f:
    data_iter = csv.reader(dest_f, 
                           delimiter = ',', 
                           quotechar = '"')
    data = [data for data in data_iter]
data_array = np.asarray(data, dtype = 'str')
print(data_array)
#print(data_array.shape)

[['PassengerId' 'Survived' 'Pclass' ... 'Fare' 'Cabin' 'Embarked']
 ['1' '0' '3' ... '7.25' '' 'S']
 ['2' '1' '1' ... '71.2833' 'C85' 'C']
 ...
 ['889' '0' '3' ... '23.45' '' 'S']
 ['890' '1' '1' ... '30' 'C148' 'C']
 ['891' '0' '3' ... '7.75' '' 'Q']]


data의 한글부분을 수치화한다.
(남자는 0, 여자는 1로)

In [2]:
quantifi_data = data_array[1:,:]

for i in range(quantifi_data.shape[0]):
    if quantifi_data[i][4]=='male':
        quantifi_data[i][4]=float(0)
    elif quantifi_data[i][4]=='female':
        quantifi_data[i][4]=float(1)

### 나이 데이터의 빈 공간을 채우는 방법을 찾아보자

방법 1. 1912년 미국의 기대수명을 이용한다. 남자의 경우 51.5살, 여자의 경우 55.9살이라는 데이터를 이용한다.
###### http://www.demog.berkeley.edu/~andrew/1918/figure2.html

In [3]:
for i in range(quantifi_data.shape[0]):
    if quantifi_data[i][5]=='':
        if quantifi_data[i][4]:
            quantifi_data[i][5]=float(51.5)
        else:
            quantifi_data[i][5]=float(55.9)
            
print(quantifi_data[5])

['6' '0' '3' 'Moran, Mr. James' '0.0' '51.5' '0' '0' '330877' '8.4583' ''
 'Q']


test 할 데이터 또한 받아온후 동일 작업을 시행한다

In [4]:
with open('test.csv','r') as dest_f:
    data_iter1 = csv.reader(dest_f, 
                           delimiter = ',', 
                           quotechar = '"')
    data1 = [data1 for data1 in data_iter1]
data_array1 = np.asarray(data1, dtype = 'str')
print(data_array1)
#print(data_array.shape)

quantifi_data1 = data_array1[1:,:]

for i in range(quantifi_data1.shape[0]):
    if quantifi_data1[i][3]=='male':
        quantifi_data1[i][3]=0
    elif quantifi_data1[i][3]=='female':
        quantifi_data1[i][3]=1
        
for i in range(quantifi_data1.shape[0]):
    if quantifi_data1[i][4]=='':
        if quantifi_data1[i][3]:
            quantifi_data1[i][4]=float(51.5)
        else:
            quantifi_data1[i][4]=float(55.9)

[['PassengerId' 'Pclass' 'Name' ... 'Fare' 'Cabin' 'Embarked']
 ['892' '3' 'Kelly, Mr. James' ... '7.8292' '' 'Q']
 ['893' '3' 'Wilkes, Mrs. James (Ellen Needs)' ... '7' '' 'S']
 ...
 ['1307' '3' 'Saether, Mr. Simon Sivertsen' ... '7.25' '' 'S']
 ['1308' '3' 'Ware, Mr. Frederick' ... '8.05' '' 'S']
 ['1309' '3' 'Peter, Master. Michael J' ... '22.3583' '' 'C']]


#### 성별과 동승자수, Pclass, Parch, 나이로 분류해보자
#### x data size는 [datanum, 5(Pclass, 성별, 나이, 동승자수, Parch)]
#### y data size는 [datanum, 1]이다

In [5]:
import tensorflow as tf

x_data = np.vstack([quantifi_data[:,2], quantifi_data[:,4], quantifi_data[:,5], quantifi_data[:,6], quantifi_data[:,7]]).T
y_data = quantifi_data[:,1].reshape(quantifi_data.shape[0],1)
x_test_data = np.vstack([quantifi_data1[:,1], quantifi_data1[:,3], quantifi_data1[:,4], quantifi_data1[:,5], quantifi_data1[:,6]]).T
test_prediction = np.zeros([quantifi_data1.shape[0],1])

X = tf.placeholder(tf.float32, [None, 5])
Y = tf.placeholder(tf.float32, [None, 1])

W1 = tf.Variable(tf.random_normal([5,5]), name='weight1')
b1 = tf.Variable(tf.random_normal([5]), name='bias1')
layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

W2 = tf.Variable(tf.random_normal([5,5]), name='weight2')
b2 = tf.Variable(tf.random_normal([5]), name='bias2')
layer2 = tf.sigmoid(tf.matmul(layer1, W2) + b2)

W3 = tf.Variable(tf.random_normal([5,1]), name='weight3')
b3 = tf.Variable(tf.random_normal([1]), name='bias3')
hypothesis = tf.sigmoid(tf.matmul(layer2, W3) + b3)

cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
#train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)
train = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y),dtype=tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(10001):
        sess.run(train, feed_dict={X: x_data,Y: y_data})
        
        if step % 1000 == 0:
            print(step, sess.run(cost, feed_dict={X: x_data,Y: y_data}), sess.run([W1, W2, W3]))
    
    h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict={X: x_data,Y: y_data})
    print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)
    
    #이제 실제 데이터를 집어넣어 예측해보자
    test_prediction = sess.run([hypothesis], feed_dict={X: x_test_data})
    print(test_prediction)
    

0 0.91751575 [array([[-1.9484181 , -0.37376827,  0.725603  ,  1.3770511 ,  0.6381118 ],
       [-0.02368974, -0.49593636, -0.33356798, -0.4294183 ,  0.32048264],
       [-1.6953728 , -0.01906802, -0.7400634 , -0.5617883 ,  0.474428  ],
       [-2.1397777 ,  0.0618189 , -0.9649807 ,  0.96291953,  0.43809226],
       [ 0.20048782, -0.3188714 ,  0.74085563, -1.0672675 , -0.34051168]],
      dtype=float32), array([[ 0.20422728, -0.6715497 , -0.00737883, -0.63082767,  0.7385151 ],
       [-0.88833815,  0.31248164, -0.01294865, -1.1963024 , -1.5075086 ],
       [ 0.16563968,  0.32791585, -1.464635  ,  0.97109383,  0.3052798 ],
       [ 0.4554284 , -0.26367173, -0.6385951 ,  0.7351263 ,  1.2799691 ],
       [-1.026015  , -1.3182825 ,  1.7057518 , -0.69019383, -1.7586313 ]],
      dtype=float32), array([[-0.9339773 ],
       [ 1.3886513 ],
       [ 0.36446625],
       [-0.0158883 ],
       [ 1.7759786 ]], dtype=float32)]
1000 0.39155975 [array([[-0.11150625,  2.310817  ,  0.38082427,  2.111567

9000 0.35388866 [array([[ -0.8399256 ,  10.342256  ,   2.2349534 ,  11.542107  ,
          5.707583  ],
       [-17.938702  ,  -6.6741056 ,   3.6178465 ,  16.400356  ,
         -6.4370875 ],
       [ -1.131868  ,  -0.48626417,  -0.09325236,   2.5037782 ,
          0.06990888],
       [ -3.648281  ,  -0.76698315,  -1.0426425 , -18.046724  ,
         -0.43233988],
       [  3.3356335 ,   1.2663313 ,   0.07206093, -14.251863  ,
         -0.49002218]], dtype=float32), array([[-41.416416  ,  17.534664  ,  31.697319  , -13.45511   ,
         14.79224   ],
       [  2.669828  ,  -3.0523145 , -15.754129  , -27.071142  ,
        -11.359682  ],
       [-13.653023  ,  -0.7650923 ,  26.069883  ,   1.2600045 ,
          8.426976  ],
       [ -5.14412   ,   7.628595  ,  10.852313  ,  -1.4245237 ,
          6.666577  ],
       [ -0.43980893, -13.733649  ,   0.13743038,   0.5214237 ,
         -6.0822644 ]], dtype=float32), array([[-5.2240257],
       [ 3.6943681],
       [-3.9347372],
       [-8.33894

test_prediction의 값을 0과 1로 정규화 시켜준다

In [6]:
for i in range(len(test_prediction[0])):
    if test_prediction[0][i] > 0.5:
        test_prediction[0][i] = 1
    else:
        test_prediction[0][i] = 0

print(test_prediction)

[array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.]

예측값들을 csv파일로 저장한다

In [7]:
f = open('prediction4.csv', 'w', encoding='utf-8', newline='')
wr = csv.writer(f)
wr.writerow(["PassengerId", "Survived"])
for i in range(len(test_prediction[0])):
    wr.writerow([892+i, int(test_prediction[0][i])])
    
f.close()